# Saving data to Google Sheets


This snippet uses the open-source [gspread](https://github.com/burnash/gspread) library for interacting with Sheets.

# Importing data from Google Sheets

In [50]:
#SONIO 
#Sandra Rojo script sandrarojo@sonio.ia

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

file = gc.open('NS')
worksheet = file.worksheet("Cleaned_All")

# get_all_records.
rows = worksheet.get_all_records()

# Convert to a pandas DataFrame.
import pandas as pd
raw = pd.DataFrame.from_records(rows )
#print(raw.head(5))

In [2]:
#TEST
#Changing the ID to an anomized ID
import numpy as np
tempdf= raw[['Nom_Fichier']].copy()
#Empty character empty cells with NaN
tempdf = tempdf.replace(r'^\s*$', np.NaN, regex=True)

In [3]:
#Create anonymyzed index
# creating bool series True for NaN values
bool_series = pd.notnull(tempdf["Nom_Fichier"])

# Independent df with new Ids
unicos = pd.DataFrame(tempdf[bool_series])
unicos['Contador'] = unicos.reset_index().index
unicos['Contador'] = 'NS-' + unicos['Contador'].astype(str)
#print(unicos)


In [4]:
#TEST
#If the name of the file is the same than the name of file in temp df with unique id, then cc the newID

tempdf['new']= tempdf.Nom_Fichier.isin(unicos.Nom_Fichier)
tempdf['new'][tempdf.Nom_Fichier.isin(unicos.Nom_Fichier)] = unicos['Contador']

# Replace False for NaN
tempdf['new'].replace(False, np.NaN, inplace=True)

#Forwardfill of the column
tempdf["new"] = tempdf["new"].ffill()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [5]:
#If the name of the file is the same than the name of file in temp df with unique id, then cc the newID
#The same but for original file
raw['Sonio_ID']= raw.Nom_Fichier.isin(unicos.Nom_Fichier)
raw['Sonio_ID'][raw.Nom_Fichier.isin(unicos.Nom_Fichier)] = unicos['Contador']

# Replace False for NaN
raw['Sonio_ID'].replace(False, np.NaN, inplace=True)

#FFill of the id column
raw['Sonio_ID'] = raw['Sonio_ID'].ffill()
print(raw.head(n=6))

         Nom_Fichier Sonio_ID  ... Mensurations  
0  hypospadias-1.txt     NS-0  ...               
1                        NS-0  ...               
2  hypospadias-2.txt     NS-1  ...  PCN ‡ 34 cm  
3                        NS-1  ...               
4                        NS-1  ...               
5                        NS-1  ...               

[6 rows x 27 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
# Matching the signs
#Read the thesaurus
#worksheet thesaurus= Thesaurus_Malfo

worksheet_th = file.worksheet("Thesaurus_Malfo")

# get_all_records.
rec_th = worksheet_th.get_all_records()

# Convert to a pandas DataFrame.
import pandas as pd
Thesaurus_Malfo = pd.DataFrame.from_records(rec_th)
#print(Thesaurus_Malfo.head(5))

#Remove columns we do not need
#list(Thesaurus_Malfo.columns)

columns = ['','accents','no accents']
Thesaurus_Malfo.drop(columns, inplace=True, axis=1)


In [51]:
#Thesaurus_Malfo.head()
raw.head()

,Nom_Fichier,Sonio_ID,Nom_Maladie,Id_Orpha,Symptome,correspondance,Nom_HPO,Id_HPO,Examen_Detection,SA,Sexe_Filleoetus,Text_Facteur_Risque,Facteur_Risque,Id_Facteur_Risque,Texte_libre_Tests_Génomiques,Test_Génomiques,Id_Tests_Génomiques,Statut_Diag,Diag_Texte_Libre,Autre,Poids,Antecedents GOB,Antecedents Famile,Serologie,Age_Mere,Mensurations,
0,hypospadias-1.txt,NS-0,Hypospadias pénien antérieur,,Hypospadias pénien antérieur,hypospadias pénien antérieur,Hypospadias,47,Examen clinique 6 mois,naissance à terme,Garcon,,Garcon,SonioRisk:072,,,#N/A,,,,3 310 g,G4P3,Grossesse spontanÈe sans particularitÈ.\n Pas ...,,,,
1,,NS-0,,,coudure et trouble de la rotation du penis,,,#N/A,,,,,,#N/A,,,#N/A,,,,,,,,,,
2,hypospadias-2.txt,NS-1,,,,,#N/A,#N/A,EXAMEN CLINIQUE 22/01/2020,,Garcon,,Garcon,SonioRisk:072,,,#N/A,,,Séroconversion au 1er trimestre de grossesse d...,3365 g,,une soeur ainée née en 2017.,,,PCN ‡ 34 cm,
3,,NS-1,,,hypospadias balanique,hypospadias balanique,Hypospadias,47,EXAMEN CLINIQUE 22/01/2020,,,,Cytomegalovirus,SonioRisk:003,,,#N/A,,,Poids: 3365 g,,,,,,,
4,,NS-1,,,halo hyperéchogene périventriculaire,halo hyperéchogene périventriculaire,#N/A,#N/A,ECHO 24 SA,,,,,#N/A,,,#N/A,,,,,,,,,,


In [48]:
##Matching 
#Make the match in between the df of Cleaned_All and the Thesaurus signs
#list(raw.columns)


In [47]:

#tempy1= raw[['correspondance','Nom_HPO','Id_HPO']].copy()
tempy1[tempy1['correspondance'].str.contains('ventricules')]


,correspondance,Nom_HPO,Id_HPO
104,"dilatation tri-ventriculaire (v1, v2 et v3) st...",#N/A,#N/A
106,discrete dilatation des ventricules lateraux p...,#N/A,#N/A
689,asymetrie ventriculo-arterielle avec nette pre...,#N/A,#N/A


In [14]:
tempy1.head(n=50)

,correspondance,Nom_HPO,Id_HPO
0,hypospadias pénien antérieur,#N/A,#N/A
1,,#N/A,#N/A
2,,#N/A,#N/A
3,hypospadias balanique,#N/A,#N/A
4,halo hyperéchogene périventriculaire,#N/A,#N/A
5,augmentation modérée de la taille des cornes t...,#N/A,#N/A
6,hypospade tres distal,#N/A,#N/A
7,verge de taille normale,#N/A,#N/A
8,2 testicules en position intra scrotale,#N/A,#N/A
9,prepuce en tablier de sapeur tres excedentaire...,#N/A,#N/A


In [ ]:
#Make the match between the HPOs and the genes associated using the PCAN library or something excitign with python



In [49]:
# Update googlespread sheet
#!pip install --upgrade gspread
# gspread version 3.7 needed

worksheet.update([raw.columns.values.tolist()] + raw.values.tolist())

{'spreadsheetId': '1qv9LPAfYoUhQTE1-XPRzAZXtaKmoHbDSVDKC9vOSEOU',
 'updatedCells': 27918,
 'updatedColumns': 27,
 'updatedRange': 'Cleaned_All!A1:AA1034',
 'updatedRows': 1034}